In [2]:
import torch as tr
import numpy as np
# from CSW import *

from matplotlib import pyplot as plt
%matplotlib inline

%load_ext autoreload
%reload_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
from CSWSEM import *
from cswRNN import *

TensorFlow Version: 2.3.1
TensorFlow Version: 2.3.1


In [3]:
# task
pr = 1.0
taskL = [CSWTask(pr),CSWTask(1-pr)]
eval_pathL = [
  [10,0,1,4,5],       
  [11,0,1,3,5],
  ]
xeval = taskL[0].format_Xeval(eval_pathL)
xeval = tr.tensor(xeval)


In [62]:
nsch = 2
seed = np.random.randint(99)
model = SEMrep(stsize=40,nschemas=nsch,seed=seed)

## task params
neps = 200
block_len = 40
learn_rate = 0.1

## train setup
softmax = lambda ulog: tr.softmax(ulog,-1)
lossop = tr.nn.CrossEntropyLoss()

# record
lossL,peL = [],[]
sch_use = np.zeros([nsch,neps])
tdim,sm_dim=5,12
ysm = -np.ones([nsch,2,neps,tdim,sm_dim])

## train loop
task_int = 0
for ep in range(neps):
  # curriculum: select task
  if ep == 160:
    block_len = 1
  if ep%block_len==0:
    task_int = (task_int+1)%2
    task = taskL[task_int]
    filler_id = 10+task_int 
  # generate data
  path = task.sample_path()
  xtrain,ytrain = task.dataset_onestory_with_marker(path=path,filler_id=filler_id,depth=1)
  # select net
  net = model.select_net(xtrain,ytrain) 
  sch_use[net.seed-seed,ep] = 1
  # eval
  for sch_idx,xev in enumerate(xeval):
    for net_idx,net in enumerate(model.schlib):
      ysm_t = softmax(net(xev)).detach().numpy()
      ysm[net_idx,sch_idx,ep] = ysm_t
  # forward prop train data
  yhat = net(xtrain)
  # back prop
  pe = model.calc_PE(yhat,ytrain)
  peL.append(pe)
  loss = 0
  optiop = tr.optim.Adam(net.parameters(), lr=learn_rate)
  optiop.zero_grad()
  for tstep in range(len(xtrain)):
    loss += lossop(yhat[tstep].unsqueeze(0),ytrain[tstep])
    loss.backward(retain_graph=True)
  optiop.step()
  lossL.append(loss.detach().numpy())
  